In [ ]:
# STEP 1: Import & setup

import os
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from PIL import Image
from tqdm import tqdm
import random

# DL
import torch
import torch.nn as nn
from torchvision import models, transforms
from torch.utils.data import Dataset, DataLoader

# Device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

In [ ]:
# STEP 2: Paths & global config

DATA_ROOT = "/kaggle/input/lisa-traffic-light-dataset"
WORK_DIR = "/kaggle/working"

DAY_TRAIN = os.path.join(DATA_ROOT, "dayTrain")
NIGHT_TRAIN = os.path.join(DATA_ROOT, "nightTrain")

IMG_SIZE = 64
BATCH_SIZE = 32
NUM_EPOCHS = 25
NUM_CLASSES = 3
SEED = 42

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)

print("DATA_ROOT exists:", os.path.exists(DATA_ROOT))

In [ ]:
# STEP 3: Label mapping

LABEL_MAP = {
    "stop": "RED",
    "warning": "YELLOW",
    "go": "GREEN",
    "goLeft": "GREEN",
    "goForward": "GREEN"
}

CLASS_TO_IDX = {"RED": 0, "YELLOW": 1, "GREEN": 2}
IDX_TO_CLASS = {v: k for k, v in CLASS_TO_IDX.items()}

LABEL_MAP, CLASS_TO_IDX

In [ ]:
# STEP 4: Load input image frame

img_path = os.path.join(
    DAY_TRAIN,
    "dayClip1",
    "frames",
    "dayClip1--00001.jpg"
)

frame = cv2.imread(img_path)
frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

plt.imshow(frame)
plt.title("Input camera frame")
plt.axis("off")

In [ ]:
# STEP 5: Load annotation CSV

csv_path = os.path.join(
    DAY_TRAIN,
    "dayClip1",
    "frameAnnotationsBOX.csv"
)

df = pd.read_csv(csv_path, sep=";")
df.head()

In [ ]:
# STEP 6: Get bounding boxes for this frame

frame_name = "dayClip1--00001.jpg"
rows = df[df["Filename"] == frame_name]
rows

In [ ]:
# STEP 7: Crop ROIs

img = Image.open(img_path).convert("RGB")

rois, labels = [], []

for _, r in rows.iterrows():
    label = r["Annotation tag"]
    if label not in LABEL_MAP:
        continue
    
    crop = img.crop((
        int(r["Upper left corner X"]),
        int(r["Upper left corner Y"]),
        int(r["Lower right corner X"]),
        int(r["Lower right corner Y"])
    )).resize((IMG_SIZE, IMG_SIZE))
    
    rois.append(crop)
    labels.append(LABEL_MAP[label])

plt.figure(figsize=(10,3))
for i, roi in enumerate(rois):
    plt.subplot(1, len(rois), i+1)
    plt.imshow(roi)
    plt.title(labels[i])
    plt.axis("off")

In [ ]:
# STEP 8: CNN inference (placeholder)

predicted_states = labels  # temporary, replace with model prediction
predicted_states

In [ ]:
# STEP 9: Rule-based planner

def stopping_distance(v, a=3.0, t=0.7):
    return (v*v)/(2*a) + v*t

def planner(light, v, d):
    if light == "RED":
        return "STOP"
    if light == "GREEN":
        return "GO"
    if light == "YELLOW":
        return "STOP" if d > stopping_distance(v) else "PROCEED"

speed = 10.0      # m/s
distance = 25.0   # m

for state in predicted_states:
    print(state, "→", planner(state, speed, distance))